- TODO: Parallelize sharp wave property computation

# Imports and definitions

In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
import numpy as np
import pandas as pd
import json
from datetime import datetime

In [13]:
import ecephys_analyses as ea
from ecephys_analyses.load import load_sr_chans
from sglxarray import load_trigger
from ecephys.xrsig import get_kcsd
from ecephys.signal.sharp_wave_ripples import detect_sharp_waves_by_value, get_durations, get_midpoints, get_sink_amplitudes, get_sink_integrals
from ecephys.utils import store_df_h5

## Detect SPWS, accounting for drift

In [14]:
def get_epoch_spws(csd, spw_params, epoch_start, epoch_end, sr_chans):
    if not sr_chans:
        return pd.DataFrame()
    
    sr_csd = csd.sel(time=slice(epoch_start, epoch_end), channel=sr_chans)
    
    spws = detect_sharp_waves_by_value(
        sr_csd.time.values,
        sr_csd.values,
        spw_params["detection_threshold"],
        spw_params["boundary_threshold"],
        spw_params["minimum_duration"],
    )

    if not spws.empty:
        spws["duration"] = get_durations(spws)
        spws["midpoint"] = get_midpoints(spws)
        spws["sink_amplitude"] = get_sink_amplitudes(spws, sr_csd.time.values, sr_csd.values) * (
            1e-6
        )  # Scale to mA/mm
        spws["sink_integral"] = (
            get_sink_integrals(spws, sr_csd.time.values, sr_csd.fs, sr_csd.values) * (1e-6) * (1e3)
        )  # Scale to mA * ms

        spws["sr_chans"] = [sr_chans] * len(spws)
    
    return spws

In [15]:
def get_file_spws(bin_path, sr_chans_path, spw_path, params_path):
    with open(params_path) as params_file:
        spw_params = json.load(params_file)
        csd_params = spw_params["csd_params"]

    sig = load_trigger(bin_path, csd_params['csd_channels'])
        
    csd = get_kcsd(
        sig,
        np.arange(0, len(sig.channel)) * csd_params['electrode_pitch'],
        drop_chans=csd_params["channels_omitted_from_csd_estimation"],
        do_lcurve=False,
        gdx=csd_params["gdx"],
        R_init=csd_params["R"],
        lambd=csd_params["lambd"],
    ).swap_dims({'pos': 'channel'})

    sr_chans_df = load_sr_chans(sr_chans_path)
    spws_by_epoch = list()
    for epoch in sr_chans_df.itertuples():
        spws_by_epoch.append(
            get_epoch_spws(
                csd, spw_params, epoch.start_time, epoch.end_time, epoch.sr_chans
            )
        )

    spws = pd.concat(spws_by_epoch)
    metadata = dict(
        spw_params=spw_params,
        csd_params=csd_params,
        file_start=str(sig.datetime.values.min()),
    )
    metadata.update(spws.attrs)

    store_df_h5(spw_path, spws, **metadata)

In [16]:
def get_experiment_spws(subject, experiment, probe):
    #hpc_chans = ea.get_channels(subject, experiment, probe, "hippocampus")
    #internal_reference = ea.get_channels(subject, experiment, probe, "internal_reference")
    
    bin_paths = ea.get_sglx_style_datapaths(subject=subject, experiment=experiment, condition="all", ext="lf.bin")
    sr_chans_paths = ea.get_sglx_style_datapaths(subject=subject, experiment=experiment, condition="all", ext="sr_chans.csv")
    spw_paths = ea.get_sglx_style_datapaths(subject=subject, experiment=experiment, condition="all", ext="spws.h5")
    params_path = ea.get_datapath(file="sharp_wave_detection_params.json", subject=subject, experiment=experiment)

    for bin_path, sr_chans_path, spw_path in zip(bin_paths, sr_chans_paths, spw_paths):
        #get_file_spws(bin_path, sr_chans_path, spw_path, params_path, hpc_chans, internal_reference)
        get_file_spws(bin_path, sr_chans_path, spw_path, params_path)
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"{current_time}: Finished {str(bin_path)}")

# Run automated pipeline

In [ ]:
get_experiment_spws("Luigi", "sleep-homeostasis", "imec0")

nChan: 385, nFileSamp: 18000070


/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/ipykernel_launcher.py:12: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['sr_chans'], dtype='object')]

  if sys.path[0] == '':


16:57:23: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX9-Luigi/6-20-2021/6-20-2021_g0/6-20-2021_g0_imec0/6-20-2021_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 18000070
17:10:09: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX9-Luigi/6-20-2021/6-20-2021_g0/6-20-2021_g0_imec0/6-20-2021_g0_t1.imec0.lf.bin
nChan: 385, nFileSamp: 18000070
17:22:34: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX9-Luigi/6-20-2021/6-20-2021_g0/6-20-2021_g0_imec0/6-20-2021_g0_t2.imec0.lf.bin
nChan: 385, nFileSamp: 18000070
